# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [2]:
# Data
import numpy as np
import pandas as pd
from collections import defaultdict

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msn
from wordcloud import WordCloud

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# Regression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

# Classification
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error


# Hide warnings
import warnings
warnings.filterwarnings('ignore')


In [5]:
df = pd.read_csv("data/train.csv")
df.sample(5)

,App Name,Category,Reviews,Size,Installs,Type,Price,Content Rating,Genre,Last Updated,Current Version,Android Version,Y
2512,trivago: Hotels & Travel,TRAVEL_AND_LOCAL,219848,Varies with device,"50,000,000+",Free,0,Everyone,Travel & Local,"August 2, 2018",Varies with device,Varies with device,4.2
7359,Mini DV,AUTO_AND_VEHICLES,12,19M,"5,000+",Free,0,Everyone,Auto & Vehicles,"May 29, 2018",1.27,4.0 and up,3.5
3203,VPN Free - Betternet Hotspot VPN & Private Bro...,TOOLS,799206,8.9M,"10,000,000+",Free,0,Everyone,Tools,"August 1, 2018",4.3.0,4.1 and up,4.5
7999,EP Horlogerie,LIFESTYLE,18,24M,"1,000+",Free,0,Everyone,Lifestyle,"March 28, 2018",4.0.2,4.0.3 and up,4.9
3558,Dark Infusion Substratum Theme for Android N & O,PERSONALIZATION,133,59M,"1,000+",Paid,$2.49,Everyone,Personalization,"August 3, 2018",15.6,7.0 and up,4.8
